In [ ]:
#@title Quotation Generator MY
# UPload Template And DB file Before Proceed
# imports

try:
  import pandas as pd
  from pathlib import Path
  import openpyxl
  import os
  from openpyxl import load_workbook
  import string
  from datetime import date
  from openpyxl.styles import Border, Side
  from PIL import Image
  from openpyxl.drawing.spreadsheet_drawing import AbsoluteAnchor
  from openpyxl.drawing.xdr import XDRPoint2D, XDRPositiveSize2D
  from openpyxl.utils.units import pixels_to_EMU
  import sys
except Exception as E:
  %pip install pandas==1.5.3
  %pip install openpyxl==3.1.2
finally:
  import pandas as pd
  from pathlib import Path
  import openpyxl
  import os
  from openpyxl import load_workbook
  import string
  from datetime import date
  from openpyxl.styles import Border, Side
  from PIL import Image
  from openpyxl.drawing.spreadsheet_drawing import AbsoluteAnchor
  from openpyxl.drawing.xdr import XDRPoint2D, XDRPositiveSize2D
  from openpyxl.utils.units import pixels_to_EMU
  import sys


sys.set_int_max_str_digits(10000)

class openImage(openpyxl.drawing.image.Image):
    def __init__(self, img,height=None,width=None):

        self.ref = img
        mark_to_close = isinstance(img, str)
        self.image = openpyxl.drawing.image._import_image(img)
        if height is not None and width is not None:
          self.height=height
          self.width=width
        else:
          self.height,self.width=self.image.size
        self.image.thumbnail((self.width,self.height),Image.LANCZOS)

        try:
            self.format = self.image.format.lower()
        except AttributeError:
            self.format = "png"
        if mark_to_close:
            # PIL instances created for metadata should be closed.
            self.image.close()

# create Images Dir and Upload Images
Path('Images/').mkdir(parents=True, exist_ok=True)
input("Press enter after uploading Images")
# upload Quote and templates in Root Folder
input("press enter after uploading .xlsx files")
# UserName and Customer_Name
Customer_Name = str(input("enter Customer Name:"))
UserName      = str(input("enter UserName:"))
rowheight     = 20
# sheet configs
autofit_columns = ["B"]
common_row_num               = 2
pagesetupArgs             = dict(paperSize=9)
selection_column_name     = "Selected"
default_row_helght_image  = 80
default_row_height_single = 90
margin                    = 5
minrow_height             = 20
default_image_widht       = 50
default_image_column_width= 23
common_image_x_pos        = 795
common_image_y_pos        = 195
imageexcelratio           = .78
colmargin                 = 5
common_image_x_size       = 8
alignment                 = 2
adjustheightsinglecol     = 5
adjust_y_pos              = 2
required_columns          = [
    "Catalog Code",
    "Description",
    "Image",
    "Rate FOB Mumbai per pc",
    "Inner Qty",
    "Outer Qty",
    "Outer L Cm",
    "Outer B Cm",
    "Outer H Cm",
    "CBM",
    "Gross Wt",
]
outputfile_name              = "{}_{}_{}_quote.xlsx".format(Customer_Name, UserName,date.today())

inputfilename                = "Quote.xlsx"
doublerowheight              = 40
outputtemp                   = "Template.xlsx"
keys_to_remove_in_image_cols = ["Image"]
template_sheet_name          = "Sheet1"
skip_row_num                 = 6
Base_Dir                     = Path().absolute()
Image_folder                 = Base_Dir.joinpath("Images")
working_file_name            = "{}_{}_{}_working.xlsx".format(Customer_Name,UserName,date.today())
#get filter rows acroding to the selection column
def get_filterd_and_sorted_data():
    file = pd.read_excel(os.path.join(Base_Dir, inputfilename))
    df   = file[file[selection_column_name] == 1]
    df   = df[required_columns]
    df   = df.sort_values("Image")
    df.to_excel(working_file_name)
    return df

# geting indecs from list
def indices(lst, item):
    return [i for i, x in enumerate(lst) if x == item]

# detect sets from sheet
def get_merge_ranges(wb, sheet):
    alphas       = list(string.ascii_uppercase)
    rows         = sheet[alphas[get_filterd_and_sorted_data().columns.to_list().index("Image")]]
    rows         = [x.value for x in rows]
    merge_ranges = dict((x, indices(rows, x)) for x in set(rows) if rows.count(x) > 1)
    unique_items = dict((x, indices(rows, x)) for x in set(rows) if rows.count(x) == 1)
    merge_ranges.update(unique_items)
    formetted_merge_ranges = {}
    for k, v in merge_ranges.items():
        if k is not None:
            if k.strip() not in keys_to_remove_in_image_cols:
                formetted_merge_ranges[k] = v

    sorted_ranges = [sorted(x) for x in list(formetted_merge_ranges.values())]
    excel_formet  = []
    for x in sorted_ranges:
        str = "{}{}:{}{}"
        if len(x) > 1:
            str = str.format(
                alphas[get_filterd_and_sorted_data().columns.to_list().index("Image")],
                x[0] + 1,
                alphas[get_filterd_and_sorted_data().columns.to_list().index("Image")],
                x[-1]+1,
            )
        else:
            str = "{}{}".format(
                alphas[get_filterd_and_sorted_data().columns.to_list().index("Image")],
                x[0] + 1,
            )
        excel_formet.append(str)
    def Mysort(s):
      return int(s.split(":")[0][1:])
    return sorted(excel_formet,key=Mysort)

# loading quotation template
# loading quotation template
def load_template():
    wb     = load_workbook(outputtemp)
    sheet  = wb["Sheet1"]
    alphas = list(string.ascii_uppercase)
    for x, l in enumerate(get_filterd_and_sorted_data().columns.to_list()):
        sheet[alphas[x] + str(skip_row_num)] = l

    for x in range(len(get_filterd_and_sorted_data())):
        sheet.insert_rows(skip_row_num + x + 1)
        for i, l in enumerate(get_filterd_and_sorted_data().columns.to_list()):
            rowdata = get_filterd_and_sorted_data().iloc[x].values
            sheet[alphas[i] + str(skip_row_num + x + 1)] = rowdata[i]
    borderrange=[skip_row_num,skip_row_num+len(get_filterd_and_sorted_data())]
    sheet[alphas[common_row_num-1] + str(2)]=date.today()
    sheet[alphas[common_row_num-1] + str(3)]=Customer_Name
    sheet[alphas[common_row_num-1] + str(4)]=UserName
    return wb, sheet,[borderrange,len(get_filterd_and_sorted_data().columns.to_list())]

# loading images to excel file
def load_images(wb, sheet):
    alphas            = list(string.ascii_uppercase)
    sheet.column_dimensions[
        alphas[get_filterd_and_sorted_data().columns.to_list().index("Image")]
    ].width           = default_image_column_width
    v_centerAlignment = openpyxl.styles.Alignment(
        horizontal="center", vertical="center", wrapText=True
    )
    common_image=common_image_y_pos
    rowheight=minrow_height
    for xs in get_merge_ranges(wb, sheet):
        v_centerAlignment = openpyxl.styles.Alignment(
            horizontal="center", vertical="center", wrapText=True
        )
        if len(xs.split(":")) > 1:
            sheet.merge_cells(xs)
        if type(sheet[xs]) == tuple:
            if sheet[xs][0][0].value.strip() not in keys_to_remove_in_image_cols:
                try:
                    row_num                                   = [int(sq[1:]) for sq in xs.split(":")]
                    img        = openImage(
                    Path.joinpath(Image_folder, sheet[xs][0][0].value)
                    )
                    p2e = pixels_to_EMU
                    h, w = img.height, img.width
                    if w > (default_image_column_width*common_image_x_size):
                      aspect=w/h
                      size = XDRPositiveSize2D(p2e(default_image_column_width*common_image_x_size), p2e((default_image_column_width*common_image_x_size)*aspect))
                      h=(default_image_column_width*common_image_x_size)*aspect
                    else:
                      size = XDRPositiveSize2D(p2e(w), p2e(h))
                    rowheight=minrow_height
                    colrowheight=(((row_num[1]-row_num[0])+1)*minrow_height)
                    if colrowheight<(h*imageexcelratio):
                       diff=(h*imageexcelratio)-colrowheight
                       singlediff=diff/((row_num[1]-row_num[0])+1)
                       rowheight+=singlediff
                       rowheight+=colmargin
                    increaserange=row_num
                    increaserange[1]+=1
                    heightincre=False
                    for q in range(*increaserange):
                       if rowheight!=minrow_height:
                          sheet.row_dimensions[int(q)].height = rowheight
                          heightincre=True
                       else:
                          sheet.row_dimensions[int(q)].height = rowheight
                    
                    if heightincre==False:
                       center=(colrowheight-(h*imageexcelratio))/2
                       position = XDRPoint2D(p2e(common_image_x_pos), p2e(common_image+center))
                       common_image=common_image+(rowheight*((row_num[1]-row_num[0])+2.5))
                    else:
                       center=((rowheight*((row_num[1]-row_num[0])+1))-(h*imageexcelratio))/2
                       position = XDRPoint2D(p2e(common_image_x_pos), p2e((common_image-((h*imageexcelratio)*.15))+center))
                       common_image=common_image+(rowheight*((row_num[1]-row_num[0])+1.5))
                    img.anchor = AbsoluteAnchor(pos=position, ext=size)
                    sheet.add_image(img)
                    sheet[xs][0][0].alignment = v_centerAlignment
                except FileNotFoundError as E:
                  print(E)
        else:
            if sheet[xs].value.strip() not in keys_to_remove_in_image_cols:
              try:
                  row_num                                   = list(xs)[1:]
                  row_num                                   = "".join(row_num)
                  img                                       = openImage(Path.joinpath(Image_folder, sheet[xs].value))
                  p2e                                       = pixels_to_EMU
                  h, w = img.height, img.width
                  aspect=w/h
                  size   = XDRPositiveSize2D(p2e(default_image_column_width*common_image_x_size), p2e((default_image_column_width*common_image_x_size)*aspect))
                  center = ((default_image_column_width*common_image_x_size)*aspect)/2
                  center -= (((size.height)/9525)*imageexcelratio)/2
                  center += colmargin/2
                  sheet.row_dimensions[int(row_num)].height = ((default_image_column_width*common_image_x_size)*aspect)+colmargin
                  position = XDRPoint2D(p2e(common_image_x_pos), p2e(common_image+center))
                  common_image=common_image+((default_image_column_width*common_image_x_size)*aspect)+colmargin
                  img.anchor = AbsoluteAnchor(pos=position, ext=size)
                  sheet.add_image(img)
                  sheet[xs].alignment = v_centerAlignment
              except FileNotFoundError as E:
                print(E)
    return wb, sheet, "Images Added OK"

# page setupargs
def setpage_layout(wb, sheet, status, pagesetupargs={}):
  for k, v in pagesetupargs.items():
    setattr(sheet.page_setup, k, v)
  return wb, sheet, "Page Layout Set OK"


# autofit columns
def autofit_single_columns(columns, wb, sheet):
    for column in sheet.columns:
        if column[0].column_letter in columns:
            max_length = 0
            column_letter = column[0].column_letter
            for cell in column:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = (max_length + 2) * 1.2
    sheet.column_dimensions[column_letter].width = adjusted_width
    return "AutoFit Done {}".format(",".join(columns)),wb


# running main function
def run(*args):
    print("status {}".format(args[-1][-1]))
    msg,wb=autofit_single_columns(autofit_columns, args[0][0], args[0][1])
    args[0][0].save(outputfile_name)
    print("file generated name {} in location {}".format(outputfile_name, Base_Dir))


def set_border(ws, cell_range):
    thin = Side(border_style="thin", color="000000")
    for row in ws[cell_range]:
        for cell in row:
            cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    return ws

if __name__ == "__main__":
    wb, sheet,borderrange = load_template()
    sheet=set_border(sheet,"A{}:{}{}".format(str(borderrange[0][0]),string.ascii_uppercase[borderrange[1]-1],str(borderrange[0][1])))
    wb, sheet, status = load_images(wb, sheet)
    run(setpage_layout(wb, sheet, status, pagesetupargs=pagesetupArgs))
